In [ ]:
#libraries
from bs4 import BeautifulSoup
import requests, datetime, time
import csv

# adjusting the link and the header
#using mechanical keyboards as the keyword 
url = "https://www.amazon.eg/s?k=mechinical+keyboard&language=en_AE&crid=1UBHQ81JZMC5G&sprefix=mechinical+keyboard%2Caps%2C206&ref=nb_sb_noss_1"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Accept-Encoding": "gzip, deflate", "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1", "Connection": "close", "Upgrade-Insecure-Requests": "1"}

#Function to pull html code from page
def get_data(url):
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")

    # getting the html code of the webpage
    title = soup.find_all("span", "a-size-base-plus a-color-base a-text-normal")
    price = soup.find_all("span", {"class": "a-price-whole"})

    # making a list for the data to be saved in
    global title_list, price_list
    title_list = [titlex.text.strip() for titlex in title]
    price_list = [pricex.text.strip()[:-1] for pricex in price]

    return soup


#Function to extract url from pages, and to stop if the next button is disabled
def get_nextpage(soup):
    pages = soup.find('span', {'class': "s-pagination-strip"})
    if not pages.find({'class': 's-pagination-item s-pagination-next s-pagination-disabled'}):
        try:
            url = 'https://www.amazon.eg/-' + str(pages.find('a',
                {'class': 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator'}).get('href'))
            return url
        except AttributeError:
            pass
        
        
# using this command to make the while loop put the data from first page and to exit the small loop after that to get back to next pages
command = False

while True:
    
# if statment to make the while loop put the data from first page only once
    if not command:
        header = ['Title', 'Price']
        data = [title_list, price_list]
        
# Creating a file and storing data from the first page
        with open('AmazonWebScraperDataset.csv', 'a', newline='', encoding='UTF8') as f:
            writer = csv.writer(f)
            writer.writerow(header)
            writer.writerow(data)
        command = True
        
# Loop to store data from pages other than the first one        
    data = get_data(url)
    url = get_nextpage(data)
    if not url:
        break
        
    url = url.replace('en/en', 'en')
    data = [title_list, price_list]
    
#Opening the file and storing the new data from other pages
    with open('AmazonWebScraperDataset.csv', 'a', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)